# Milestone 2
# Renuka Mulay


Read the Automobile Data Set from Internet and Assign Columns Names.

In [89]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from scipy import stats

In [90]:
#Read the WholeSale Customer Data from Internet and Assign Columns Names.
headers = ["Channel", "Region", "Fresh" ,"Milk", "Grocery","Frozen","Detergents_Paper","Delicassen"]
originalData = pd.read_csv('https://raw.githubusercontent.com/renukamulay/NoteBookShare/master/data/Wholesale_customers_data.csv', 
                          header=0, names=headers, na_values="0")

#Select a subset of data for analysis.
subsetData = originalData[["Channel", "Region", "Fresh" ,"Milk", "Grocery","Frozen","Detergents_Paper","Delicassen"]].copy()

Remove the Outliers using the Z Normalization. If the absolute value of the score is > 3 then the whole row is remove.

In [91]:
# Remove Outlier based on the Z Score.
# Outliers are removed based on the z score. If Z Score > 3 then the whole row is removed.
print(f"""Shape before removing the outliers {subsetData.shape}""")
subsetData = subsetData[np.abs(stats.zscore(subsetData) < 3).all(axis=1)]
print(f"""Shape after removing the outliers {subsetData.shape}""")

Shape before removing the outliers (440, 8)
Shape after removing the outliers (414, 8)


Replace missing numeric data (at least 1 column) with the median.[Note :: This dataset doesnt contain missing value]

In [92]:
# Validate Before changing
print(f"""Minimun value before Imputing Missing value = {subsetData.describe()['Grocery']['min']}""")

#Column Glucose_Concentration cannot have 0 value, and the ones that contain 0 value are the Missing Values.
missingValue = 0
#Replace 0 values with NaN so that Pandas ignores those values while caculating median.
subsetData.loc[(subsetData.Grocery == missingValue),'Grocery'] = np.NaN
missingValueReplacement = subsetData.median()['Grocery']

# replace the missing values(NaN) with the Median
subsetData.Grocery =  subsetData["Grocery"].fillna(missingValueReplacement)

# Validate After changing
print(f"""Minimun value after Imputing Missing value = {subsetData.describe()['Grocery']['min']}""")

Minimun value before Imputing Missing value = 3.0
Minimun value after Imputing Missing value = 3.0


Normalize (Min-Max Normalization) numeric values of Column (Frozen)

In [93]:
print(f"Before Normalization {subsetData['Frozen'].head(5)}")

# Select the Column to Min-Max Normalize
columnToNormalize = ['Frozen']

# Import the SkLearn classes for MinMax Normalization.
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(subsetData[columnToNormalize])

# Set the Normalized Columns to the Original DataSet.
subsetData['Frozen'] = pd.DataFrame(minmax_scaler.transform(subsetData[columnToNormalize]))

print(f""" After Normilization {subsetData[columnToNormalize].head(5)}""")

Before Normalization 0     214
1    1762
2    2405
3    6404
4    3915
Name: Frozen, dtype: int64
 After Normilization      Frozen
0  0.011187
1  0.102818
2  0.140878
3  0.377590
4  0.230259


C:\ProgramData\Anaconda3\envs\uwdatasci400\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Bin numeric variables. Column = Fresh.

Bins
1. 0 - 20000   Low
2. 20000 - 35000    Medium
3. 35000 - 50000   High


In [94]:
print(f"""Before Binning {subsetData['Fresh'].head(5)}""")

# Create the bins and apply pd.cut to bin the numeric data into categories.
bins = [0, 20000, 35000, 50000]
labels = ['Low', 'Medium', 'High']
subsetData['Fresh'] = pd.cut(subsetData['Fresh'], bins=bins, labels=labels)

# Set the Type to Category.
subsetData['Fresh'] = subsetData['Fresh'].astype('category')

print(f"""After Binning {subsetData['Fresh'].head(5)}""")

Before Binning 0    12669
1     7057
2     6353
3    13265
4    22615
Name: Fresh, dtype: int64
After Binning 0       Low
1       Low
2       Low
3       Low
4    Medium
Name: Fresh, dtype: category
Categories (3, object): [Low < Medium < High]


Consolidate categorical data. 

In [95]:
print(f"""Before Consolidation {subsetData.head(5)}""")
# # Convert the num of cylinders into number.
# numbers = {"num_cylinders": {"Three": 4, "six": 6, "five": 5, "eight": 8, "two": 2, "twelve": 12, "three":3 }}
# subsetData.replace(numbers, inplace=True)

# Consolidate the cateogories into 2 categories 1) Upto5Cylinders  2) MoreThan5Cylinders
consolidateCategories = {"LisnonAndOportoRegion" : [1,2],
      "OtherRegion" : [3]}
distribution = {v: k for k,vv in consolidateCategories.items() for v in vv}
subsetData['Region'] = subsetData['Region'].map(distribution).astype(pd.api.types.CategoricalDtype(categories=set(distribution.values())))
subsetData.head(5)

Before Consolidation    Channel  Region   Fresh  Milk  Grocery    Frozen  Detergents_Paper  \
0        2       3     Low  9656   7561.0  0.011187              2674   
1        2       3     Low  9810   9568.0  0.102818              3293   
2        2       3     Low  8808   7684.0  0.140878              3516   
3        1       3     Low  1196   4221.0  0.377590               507   
4        2       3  Medium  5410   7198.0  0.230259              1777   

   Delicassen  
0        1338  
1        1776  
2        7844  
3        1788  
4        5185  


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,OtherRegion,Low,9656,7561.0,0.011187,2674,1338
1,2,OtherRegion,Low,9810,9568.0,0.102818,3293,1776
2,2,OtherRegion,Low,8808,7684.0,0.140878,3516,7844
3,1,OtherRegion,Low,1196,4221.0,0.377590,507,1788
4,2,OtherRegion,Medium,5410,7198.0,0.230259,1777,5185


One-hot encode categorical data with at least 3 categories (at least 1 column).

In [96]:
# Onehot encode the column Fresh using the Pandas Method : get_dummies.
subsetData = pd.get_dummies(subsetData, columns=["Fresh"])

# Validate that the column is OneHotEncoded.
subsetData.head(5)

,Channel,Region,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Fresh_Low,Fresh_Medium,Fresh_High
0,2,OtherRegion,9656,7561.0,0.011187,2674,1338,1,0,0
1,2,OtherRegion,9810,9568.0,0.102818,3293,1776,1,0,0
2,2,OtherRegion,8808,7684.0,0.140878,3516,7844,1,0,0
3,1,OtherRegion,1196,4221.0,0.377590,507,1788,1,0,0
4,2,OtherRegion,5410,7198.0,0.230259,1777,5185,0,1,0


Remove obsolete columns. Columns Name : Channel.

In [97]:
print(f"""Before Dropping {subsetData.head(5)}""")

# Drop the column horsepower as it is not needed for further analysis.
subsetData = subsetData.drop(['Channel'], axis=1)

subsetData.head(5)


Before Dropping    Channel       Region  Milk  Grocery    Frozen  Detergents_Paper  \
0        2  OtherRegion  9656   7561.0  0.011187              2674   
1        2  OtherRegion  9810   9568.0  0.102818              3293   
2        2  OtherRegion  8808   7684.0  0.140878              3516   
3        1  OtherRegion  1196   4221.0  0.377590               507   
4        2  OtherRegion  5410   7198.0  0.230259              1777   

   Delicassen  Fresh_Low  Fresh_Medium  Fresh_High  
0        1338          1             0           0  
1        1776          1             0           0  
2        7844          1             0           0  
3        1788          1             0           0  
4        5185          0             1           0  


,Region,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Fresh_Low,Fresh_Medium,Fresh_High
0,OtherRegion,9656,7561.0,0.011187,2674,1338,1,0,0
1,OtherRegion,9810,9568.0,0.102818,3293,1776,1,0,0
2,OtherRegion,8808,7684.0,0.140878,3516,7844,1,0,0
3,OtherRegion,1196,4221.0,0.377590,507,1788,1,0,0
4,OtherRegion,5410,7198.0,0.230259,1777,5185,0,1,0
